In [1]:
# NOT WORKING
import numpy as np

In [2]:
def buyer_offered_deals(buyer_product_values, seller_product_values, pairings, epsilon):
    # get lowest seller offer
    min_seller = 0
    min_seller_current_partner = np.nan
    min_price = np.inf
    for seller_index in range(seller_product_values.size):
        try:
            seller_current_partner = np.nanargmax(pairings[:, seller_index])
            seller_current_price = pairings[seller_current_partner, seller_index]
        except ValueError:
            seller_current_partner = seller_current_price = np.nan
        # if seller had a succesful deal, use that price + epsilon
        if not np.isnan(seller_current_price) and seller_current_price+epsilon < min_price:
            min_price = seller_current_price+epsilon
            min_seller = seller_index
            min_seller_current_partner = seller_current_partner
        # if seller failed to have a deal, use their product_value as their price
        elif np.isnan(seller_current_price) and seller_product_values[seller_index] < min_price:
            min_price = seller_product_values[seller_index]
            min_seller = seller_index
            min_seller_current_partner = seller_current_partner

    # Buyers from strongest to weakest take the offer if:
    # - their current price is more than that, or
    # - their current price is NaN and they can afford it
    # However, buyer will push the offer down to the next best buyer's product_value + epsilon if it's:
    # - larger or equal to than the seller's product_value, and
    # - smaller than the min_price
    for buyer_index in range(buyer_product_values.size):
        try:
            buyer_current_partner = np.nanargmax(pairings[buyer_index, :])
            buyer_current_price = pairings[buyer_index, buyer_current_partner]
        except ValueError:
            buyer_current_partner = buyer_current_price = np.nan
        if not np.isnan(buyer_current_price) and buyer_current_price <= min_price: continue
        elif np.isnan(buyer_current_price) and buyer_product_values[buyer_index] <= min_price: continue
        # seller accepts offer, break
        if not np.isnan(min_seller_current_partner): pairings[min_seller_current_partner, min_seller] = np.nan
        if not np.isnan(buyer_current_partner): pairings[buyer_index, buyer_current_partner] = np.nan
        if (buyer_index != buyer_product_values.size-1
                and buyer_product_values[buyer_index+1]+epsilon >= seller_product_values[min_seller]
                and buyer_product_values[buyer_index+1]+epsilon < min_price):
            pairings[buyer_index, min_seller] = buyer_product_values[buyer_index+1]+epsilon
        else:
            pairings[buyer_index, min_seller] = min_price
        return

In [3]:
def seller_offered_deals(buyer_product_values, seller_product_values, pairings, epsilon):
    # get highest buyer offer
    max_buyer = 0
    max_buyer_current_partner = np.nan
    max_price = -1
    for buyer_index in range(buyer_product_values.size):
        try:
            buyer_current_partner = np.nanargmin(pairings[buyer_index, :])
            buyer_current_price = pairings[buyer_index, buyer_current_partner]
        except ValueError:
            buyer_current_partner = buyer_current_price = np.nan
        # if buyer had a succesful deal, use that price - epsilon
        if not np.isnan(buyer_current_price) and buyer_current_price-epsilon > max_price:
            max_price = buyer_current_price-epsilon
            max_buyer = buyer_index
            max_buyer_current_partner = buyer_current_partner
        # if buyer failed to have a deal, use their product_value as their price
        elif np.isnan(buyer_current_price) and buyer_product_values[buyer_index] > max_price:
            max_price = buyer_product_values[buyer_index]
            max_buyer = buyer_index
            max_buyer_current_partner = buyer_current_partner

    # Sellers from strongest to weakest take the offer if:
    # - their current price is less than that, or
    # - their current price is NaN and it's acceptible
    # However, seller will push the offer up to the next best seller's product_value - epsilon if it's:
    # - smaller or equal to than the buyer's product_value, and
    # - larger than the max_price
    for seller_index in range(seller_product_values.size):
        try:
            seller_current_partner = np.nanargmin(pairings[:, seller_index])
            seller_current_price = pairings[seller_current_partner, seller_index]
        except ValueError:
            seller_current_partner = seller_current_price = np.nan
        if not np.isnan(seller_current_price) and seller_current_price >= max_price: continue
        elif np.isnan(seller_current_price) and seller_product_values[seller_index] >= max_price: continue
        # buyer accepts offer, break
        if not np.isnan(max_buyer_current_partner): pairings[max_buyer, max_buyer_current_partner] = np.nan
        if not np.isnan(seller_current_partner): pairings[seller_current_partner, seller_index] = np.nan
        if (seller_index != seller_product_values.size-1
                and seller_product_values[seller_index+1]-epsilon <= buyer_product_values[max_buyer]
                and seller_product_values[seller_index+1]-epsilon > max_price):
            pairings[max_buyer, seller_index] = seller_product_values[seller_index+1]-epsilon
        else:
            pairings[max_buyer, seller_index] = max_price
        return

In [80]:
buyer_product_values = np.sort(np.array([2,4,5]))[::-1] # the max prices buyers are willing to pay, sorted from strong to weak
seller_product_values = np.sort(np.array([0,1,3])) # the min prices sellers are willing to accept, sorted from strong to weak
epsilon = 0.1
trials = 1000 # does 1000 of each buyer seller

# matrix of successful deals
# buyer i sells to seller j at price k
pairings = np.empty((buyer_product_values.size, seller_product_values.size,))
pairings[:] = np.nan
# pairings[0,0]=2.5
# pairings[1,1]=2.5

# Run 1000 rounds of deals
print("Starting pairings:")
print(pairings)
print("...")

total_rounds = 1
for round in range(trials):
    previous_pairings = pairings.copy()
    buyer_offered_deals(buyer_product_values, seller_product_values, pairings, epsilon)
    if ((pairings == previous_pairings) | (np.isnan(pairings) & np.isnan(previous_pairings))).all(): break
total_rounds += round
for round in range(trials):
    previous_pairings = pairings.copy()
    seller_offered_deals(buyer_product_values, seller_product_values, pairings, epsilon)
    if ((pairings == previous_pairings) | (np.isnan(pairings) & np.isnan(previous_pairings))).all(): break

print(f"Took {total_rounds+round+1} rounds to reach steady state:")
print(pairings)

Starting pairings:
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
...
Took 66 rounds to reach steady state:
[[nan 2.9 nan]
 [3.  nan nan]
 [nan nan nan]]


In [8]:
buyer_product_values = np.sort(np.random.random(np.random.randint(1,5)))[::-1] # the max prices buyers are willing to pay, sorted from strong to weak
seller_product_values = np.sort(np.random.random(np.random.randint(1,5))) # the min prices sellers are willing to accept, sorted from strong to weak
precision = 2
epsilon = 10**-precision

# matrix of successful deals
# buyer i sells to seller j at price k
pairings = np.empty((buyer_product_values.size, seller_product_values.size,))
pairings[:] = np.nan
# randomly generate pairings and random prices
buyer_permutation = np.random.permutation(np.arange(buyer_product_values.size))
seller_permutation = np.random.permutation(np.arange(seller_product_values.size))
for i in range(np.min([buyer_product_values.size, seller_product_values.size])):
    if buyer_product_values[buyer_permutation[i]] >= seller_product_values[seller_permutation[i]]:
        pairings[buyer_permutation[i], seller_permutation[i]] = np.random.uniform(
            buyer_product_values[buyer_permutation[i]],
            seller_product_values[buyer_permutation[i]]
        )

# Run 1000 rounds of deals
print("buyer_product_values", buyer_product_values)
print("seller_product_values", seller_product_values)
print("Starting pairings:")
print(pairings)
print("...")

trials = 1000
total_rounds = 1
for round in range(trials):
    previous_pairings = pairings.copy()
    buyer_offered_deals(buyer_product_values, seller_product_values, pairings, epsilon)
    if ((pairings == previous_pairings) | (np.isnan(pairings) & np.isnan(previous_pairings))).all(): break
total_rounds += round
# for round in range(trials):
#     previous_pairings = pairings.copy()
#     seller_offered_deals(buyer_product_values, seller_product_values, pairings, epsilon)
#     if ((pairings == previous_pairings) | (np.isnan(pairings) & np.isnan(previous_pairings))).all(): break

print(f"Took {total_rounds+round+1} rounds to reach steady state:")
print(pairings)

buyer_product_values [0.81358802 0.56092983 0.38200765 0.23211751]
seller_product_values [0.22757106]
Starting pairings:
[[0.76957406]
 [       nan]
 [       nan]
 [       nan]]
...
Took 2 rounds to reach steady state:
[[0.76957406]
 [       nan]
 [       nan]
 [       nan]]
